In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import math
import phe
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  StandardScaler
from phe import paillier
import bitstring
from bitstring import Bits
from secrets import token_hex
import copy
from Crypto.Cipher import DES
import pickle

%matplotlib inline

In [27]:
def cholesky_decomposition(A):
    d = A.shape[0]
    L = A.copy()
    #L = np.zeros(A.shape)
    for i in range(d-1):
        for j in range(i+1,d):
            L[i][j] = 0.0
    for j in range(d):
        for k in range(j-1):
            for i in range(j,d):
                L[i][j] -= (L[i][k]*L[j][k])
        L[j][j] = math.sqrt(L[j][j])
        for k in range(j+1,d):
            L[k][j] /= L[j][j]
    for i in range(d-1):
        for j in range(i+1,d):
            L[i][j] = 0.0
    return L

In [28]:
A = np.array([[ 6,  3,  4,  8],
       [ 3,  6,  5,  1],
       [ 4,  5, 10,  7],
       [ 8,  1,  7, 25]],dtype=np.float32)

In [29]:
L = cholesky_decomposition(A)

In [30]:
L

array([[2.4494898 , 0.        , 0.        , 0.        ],
       [1.2247448 , 2.4494898 , 0.        , 0.        ],
       [1.6329931 , 2.0412414 , 2.7080128 , 0.        ],
       [3.2659862 , 0.40824828, 0.6154576 , 3.7638633 ]], dtype=float32)

In [31]:
np.dot(L,L.T)

array([[ 6.0000005,  3.       ,  4.       ,  8.       ],
       [ 3.       ,  7.5      ,  7.       ,  4.9999995],
       [ 4.       ,  7.       , 14.166666 ,  7.8333335],
       [ 8.       ,  4.9999995,  7.8333335, 25.378788 ]], dtype=float32)

In [32]:
A = np.array([[ 6,  3,  4,  8],
       [ 3,  6,  5,  1],
       [ 4,  5, 10,  7],
       [ 8,  1,  7, 25]],dtype=np.float32)

In [33]:
L2 = np.linalg.cholesky(A)

In [34]:
L2

array([[ 2.4494898,  0.       ,  0.       ,  0.       ],
       [ 1.2247449,  2.1213202,  0.       ,  0.       ],
       [ 1.6329932,  1.4142135,  2.309401 ,  0.       ],
       [ 3.2659864, -1.4142135,  1.5877132,  3.132491 ]], dtype=float32)

In [35]:
np.dot(L2,L2.T)

array([[ 6.0000005,  3.0000002,  4.0000005,  8.000001 ],
       [ 3.0000002,  5.9999995,  5.       ,  1.0000007],
       [ 4.0000005,  5.       , 10.       ,  7.       ],
       [ 8.000001 ,  1.0000007,  7.       , 25.       ]], dtype=float32)

In [36]:
A

array([[ 6.,  3.,  4.,  8.],
       [ 3.,  6.,  5.,  1.],
       [ 4.,  5., 10.,  7.],
       [ 8.,  1.,  7., 25.]], dtype=float32)

In [37]:
def cholesky(A):
    """Performs a Cholesky decomposition of A, which must 
    be a symmetric and positive definite matrix. The function
    returns the lower variant triangular matrix, L."""
    n = len(A)

    # Create zero matrix for L
    L = np.zeros(A.shape)

    # Perform the Cholesky decomposition
    for i in range(n):
        for k in range(i+1):
            tmp_sum = sum(L[i][j] * L[k][j] for j in range(k))
            
            if (i == k): # Diagonal elements
                # LaTeX: l_{kk} = \sqrt{ a_{kk} - \sum^{k-1}_{j=1} l^2_{kj}}
                L[i][k] = np.sqrt(A[i][i] - tmp_sum)
            else:
                # LaTeX: l_{ik} = \frac{1}{l_{kk}} \left( a_{ik} - \sum^{k-1}_{j=1} l_{ij} l_{kj} \right)
                L[i][k] = (1.0 / L[k][k] * (A[i][k] - tmp_sum))
    return L

In [38]:
cholesky(A)

array([[ 2.44948974,  0.        ,  0.        ,  0.        ],
       [ 1.22474487,  2.12132034,  0.        ,  0.        ],
       [ 1.63299316,  1.41421356,  2.30940108,  0.        ],
       [ 3.26598632, -1.41421356,  1.58771324,  3.13249102]])

In [39]:
def square_root(x):
    e = [pow(4,i) for i in range(int(x)) if pow(4,i)<=x][-1]
    r = 0
    #print(e)
    while e != 0:
        if x >= r+e:
            x -= (r+e)
            r = (r >> 1) +e
        else:
            r = (r >> 1)
        e = (e >> 2)
    return r

In [40]:
square_root(100)

10

In [41]:
def poc_bitstring_class():
    binary = np.array(list(bin(255))[2:],dtype=np.bool)
    b = bitstring.Bits(float=1.23,length=32)
    print(b.bin)
    b = bitstring.Bits(float=-1.23,length=32)
    print(b.bin)
    
    a = bitstring.Bits(float=1.0,length=32)
    b = bitstring.Bits(float=-1.0,length=32)
    print(a.bin, b.bin,a.bin[:9])
    
    a = bitstring.Bits(float=1000.2090990909090909,length=32)
    b = bitstring.Bits(float=-127.0,length=32)
    print(a.bin, b.bin)
    
    a = bitstring.Bits(int=1,length=32)
    b = bitstring.Bits(int=-1,length=32)
    print(a.bin, b.bin)

In [42]:
poc_bitstring_class()

00111111100111010111000010100100
10111111100111010111000010100100
00111111100000000000000000000000 10111111100000000000000000000000 001111111
01000100011110100000110101100010 11000010111111100000000000000000
00000000000000000000000000000001 11111111111111111111111111111111


In [43]:
def poc_paillier_encryption_scheme():
    pub,priv = paillier.generate_paillier_keypair()
    n = pub.encrypt(1.25)
    m = pub.encrypt(1.25)
    print(priv.decrypt(m+n))

In [44]:
poc_paillier_encryption_scheme()

2.5


In [45]:
def poc_encryption_des():
    a = token_hex(4)
    b = token_hex(4)
    c = token_hex(4)
    
    print('Original Values : ')
    print('A : [ %s ], B : [ %s ], C : [ %s ]'%(a,b,c))
    print('--------------------------------------')
    print('First Encrypt with A and then B')
    des_in = DES.new(b, DES.MODE_ECB)
    des_in2 = DES.new(a,DES.MODE_ECB)
    c_cipher = des_in2.encrypt(des_in.encrypt(c))
    c_decrypted = des_in.decrypt(des_in2.decrypt(c_cipher))
    print('C Encrypted with A & B [ %s ]'%c_cipher)
    print('C Original : [ %s ], C Decrypted [ %s ]'%(c, c_decrypted))
    
    print('--------------------------------------')
    print('First Encrypt with B and then A')
    des_in = DES.new(b, DES.MODE_ECB)
    des_in2 = DES.new(a,DES.MODE_ECB)
    c_cipher = des_in.encrypt(des_in2.encrypt(c))
    c_decrypted = des_in2.decrypt(des_in.decrypt(c_cipher))
    print('C Encrypted with A & B [ %s ]'%c_cipher)
    print('C Original : [ %s ], C Decrypted [ %s ]'%(c, c_decrypted))
    
    print('--------------------------------------')
    print('First Encrypt with B and then B')
    des_in = DES.new(b, DES.MODE_ECB)
    c_cipher = des_in.encrypt(des_in.encrypt(c))
    c_decrypted = des_in.decrypt(des_in.decrypt(c_cipher))
    print('C Encrypted with A & B [ %s ]'%c_cipher)
    print('C Original : [ %s ], C Decrypted [ %s ]'%(c, c_decrypted))
    
    print('--------------------------------------')
    print('First Encrypt with A and then A')
    des_in2 = DES.new(a,DES.MODE_ECB)
    c_cipher = des_in2.encrypt(des_in2.encrypt(c))
    c_decrypted = des_in2.decrypt(des_in2.decrypt(c_cipher))
    print('C Encrypted with A & B [ %s ]'%c_cipher)
    print('C Original : [ %s ], C Decrypted [ %s ]'%(c, c_decrypted))

In [46]:
poc_encryption_des()

Original Values : 
A : [ 2eaa5fa0 ], B : [ bf160cee ], C : [ 01bafd12 ]
--------------------------------------
First Encrypt with A and then B
C Encrypted with A & B [ b'&\xfe\xe2\xe4\x12\xcbY)' ]
C Original : [ 01bafd12 ], C Decrypted [ b'01bafd12' ]
--------------------------------------
First Encrypt with B and then A
C Encrypted with A & B [ b'\x9f6\xef\xd9\xba3\xf2\xe6' ]
C Original : [ 01bafd12 ], C Decrypted [ b'01bafd12' ]
--------------------------------------
First Encrypt with B and then B
C Encrypted with A & B [ b'I\xae\x9a\xc68~\xf2\xcf' ]
C Original : [ 01bafd12 ], C Decrypted [ b'01bafd12' ]
--------------------------------------
First Encrypt with A and then A
C Encrypted with A & B [ b'~t\xda\x84Ql\x15W' ]
C Original : [ 01bafd12 ], C Decrypted [ b'01bafd12' ]


In [47]:
class User(object):
    def __init__(self):
        df = pd.read_csv('auto-mpg.csv')
        df = df[df['horsepower'] != '?']
        cols = df.columns.tolist()
        cols.remove('mpg')
        cols.remove('car name')
        self.X = df[cols].values.astype(np.float32)
        self.y = df['mpg'].values.astype(np.float32)
        self.X_train,self.X_test,self.y_train,self.y_test = train_test_split(self.X,self.y,train_size=.8,test_size=.2)
        sc = StandardScaler()
        self.X_train = sc.fit_transform(self.X_train)
        self.X_test = sc.transform(self.X_test)
        #print(self.X_train,self.X_test)
        self.X_train = np.around(self.X_train,2)
        self.X_test = np.around(self.X_test,2)
        self.temp = np.around(np.dot(self.X_train.T,self.X_train),2)
        print(np.around(np.dot(self.X_train.T,self.X_train),2))
        self.public_key, self.private_key = paillier.generate_paillier_keypair()
        #self.paillier_encrypt = np.vectorize(public_key.encrypt)
        #self.paillier_decrypt = np.vectorize(private_key.decrypt)
    
    def paillier_encrypt(self, A_i, b_i, precision=2):
        A_i_enc = np.zeros(A_i.shape,dtype=np.object)
        for i in range(A_i.shape[0]):
            for j in range(A_i.shape[1]):
                #print(A_i[i][j])
                A_i_enc[i][j] = self.public_key.encrypt(A_i[i][j],precision=precision)
                
        b_i_enc = np.zeros(b_i.shape,dtype=np.object)
        for i in range(b_i.shape[0]):
            b_i_enc[i] = self.public_key.encrypt(b_i[i],precision=precision)
        
        return A_i_enc, b_i_enc
    
    def paillier_decrypt(self,c_i,precision=2):
        #c = user.calculate_and_send_to_evaluater()
        A_hat, b_hat = np.empty(c_i[0].shape, dtype=np.object), np.empty(c_i[1].shape, dtype=np.object)
        for i in range(c_i[0].shape[0]):
            for j in range(c_i[0].shape[0]):
                A_hat[i][j] = self.private_key.decrypt(c_i[0][i][j])

        for j in range(c_i[1].shape[0]):
                b_hat[j] = self.private_key.decrypt(c_i[1][j])
        
        return A_hat, b_hat
        #for i in range(c_i[0].shape[0]):
        #    for j in range(c_i[0].shape[0]):
        #        print(self.private_key.decrypt(c_i[0][i][j]), end='')
        #        print('\t',end='')
        #    print()

        #for j in range(c_i[1].shape[0]):
        #        print(self.private_key.decrypt(c_i[1][j]))
    
    def calculate_and_send_to_evaluater(self):
        c = []
        mu_A = np.random.randn(len(self.y_train),len(self.y_train))
        mu_b = np.random.randn(len(self.y_train))
        #print(self.X_train.shape,self.y_train.shape)
        for x_i,y_i in zip(self.X_train,self.y_train):
            #print(x_i, y_i)
            A_i = np.dot(x_i.reshape(len(x_i), 1), x_i.reshape(len(x_i), 1).T)
            b_i = x_i * y_i
            #print(A_i.shape, b_i.shape)
            c_i = self.paillier_encrypt(A_i, b_i, precision=2)
            c.append(c_i)
        c_mu = self.paillier_encrypt(mu_A, mu_b, precision=2)
        c.append(c_mu)
        As,bs = zip(*c)
        c_final = sum(As),sum(bs)
        return c_final

In [48]:
user = User()
#c = user.calculate_and_send_to_evaluater()
#for i in range(c[0].shape[0]):
#    for j in range(c[0].shape[0]):
#        print(user.private_key.decrypt(c[0][i][j]), end='')
#        print('\t',end='')
#    print()
    
#for j in range(c[1].shape[0]):
#        print(user.private_key.decrypt(c_final[1][j]))


[[ 310.83  296.4   263.09  279.52 -163.61 -109.69 -186.52]
 [ 296.4   312.98  280.43  291.5  -174.98 -120.08 -200.26]
 [ 263.09  280.43  313.    270.08 -217.24 -135.14 -151.93]
 [ 279.52  291.5   270.08  312.95 -133.9  -101.48 -192.84]
 [-163.61 -174.98 -217.24 -133.9   312.94  105.55   78.25]
 [-109.69 -120.08 -135.14 -101.48  105.55  313.08   65.22]
 [-186.52 -200.26 -151.93 -192.84   78.25   65.22  313.14]]


In [49]:
##xt = np.zeros(user.temp.shape,dtype=np.object)
#for i in range(user.temp.shape[0]):
#    for j in range(user.temp.shape[1]):
#        b = Bits(float=user.temp[i][j],length=64)
#        print(b.bin)

In [10]:
def compute_twos_complement(binary):
    out = ''
    for i in binary:
        out += '1' if i=='0' else '0'
    #print(out)
    return Add()(out,((len(binary)-1)*'0')+'1')
        
def float_to_bin(num,size=24):
    twos_complement = True if num<0 else False
    int_p,frac_p = str(num).split('.')
    frac_p = float('0.'+frac_p)
    int_b = bin(int(int_p))[2:]
    frac_b = '.'
    for i in range(8):
        #print(frac_p)
        frac_p = frac_p*2
        t_i, t_f = str(frac_p).split('.')
        frac_b += t_i
        if t_i == '1':
            frac_p -= 1.
        #print(frac_p)
    out = int_b+frac_b
    out = out.replace('.','')
    #print(out)
    out = out[:-4]  ## Need to change to -8 in future
    out += '11111100'
    out = out.zfill(size)
    if twos_complement:
        temp_out = compute_twos_complement(out[:size-8])
    out = temp_out+out[size-8:] if twos_complement else out
    #print(out)
    return out

def int_to_bin(num):
    twos_complement = True if num<0 else False
    int_b = bin(int(num))[2:]
    if twos_complement:
        int_b = compute_twos_complement(int_b)
    return int_b

def bin_to_float(binary):
    int_p , frac_p = binary[:len(binary)-8], binary[len(binary)-8:]
    int_part = 0.0
    for i,j in enumerate(reversed(int_p[1:])):
        int_part += (int(j) * pow(2, i))
    #print(int_part)
    int_part -= int(binary[0])*pow(2,len(binary)-9) ## 15
    #print(int_part)
    pow_part = 0.0
    for i,j in enumerate(reversed(frac_p[1:])):
        pow_part += (int(j) * pow(2, i))
    pow_part -= pow(2,7)
    #print(pow_part)
    return int_part*pow(2,pow_part)

def bin_to_int(binary):
    int_part = 0
    for i,j in enumerate(reversed(binary[1:])):
        int_part += (int(j) * pow(2, i))
    int_part -= int(binary[0])*pow(2,len(binary)-1)
    return int_part

In [11]:
class Gate(object):
    pass

class LeftShift(Gate):
    def __call__(self,a,n):
        return a[n:] + '0' * n

class RightShift(Gate):
    def __call__(self,a,n):
        return '0' * n + a[:-n]

class And(Gate):
    def __call__(self,a,b):
        return a and b

class Or(Gate):
    def __call__(self,a,b):
        return a or b

class Xor(Gate):
    def __call__(self,a,b):
        return 0 if (a==1 and b==1) or (a==0 and b==0) else 1

class Not(Gate):
    def __call__(self,a):
        t = {0:1,1:0}
        return t[a]
                
class Add(Gate):
    
    def add(self,a,b,carry):
        #xor = Xor()
        #and_g = And()
        #or_g = Or()
        o1 = Xor()(a,b)
        o2 = And()(a,b)
        add = Xor()(carry,o1)
        o4 = And()(carry,o1)
        cary = Or()(o4,o2)
        return str(add), cary
    
    def __call__(self,a,b):
        carry = 0
        out = ''
        for idx,i in reversed(list(enumerate(zip(a[:16],b[:16])))):
            o,carry = self.add(int(i[0]),int(i[1]),carry)
            out += o
        #print(out)
        out = ''.join(reversed(out))
        return out + a[16:]

class Subtract(Gate):
    
    def __call__(self,a,b):
        b2 = compute_twos_complement(b[:16])
        b2 = b2 + b[16:]
        return Add()(a,b2)

class Multiplication(Gate):

    def __call__(self,a,b):
        s = '0'*16
        m = copy.copy(a[:16])
        for i in reversed(b[:16]):
            if int(i) != 0:     # when digit is one, add the intermediary product
                s = Add()(s, m)
            m = LeftShift()(m, 1)  # shift one per digit in b
        #exponent = Add()(a[16:],b[16:])
        s = RightShift()(s,4) ## Shifting left by 4 places to get exponent back to 2^-4

        return s+a[16:]
        
class Division(Gate):
    
    def __call__(self,a, b):
        reminder,reminder_prev = Subtract()(a[:16], b[:16]), copy.copy(a[:16])
        quotient = Add()('0'*len(a[:16]), ((len(a[:16])-1)*'0')+'1')
        while '1' in reminder:
            reminder_prev = copy.copy(reminder)
            reminder = Subtract()(reminder, b[:16])
            #print(bin_to_int(reminder))
            #quotient = Add()(quotient, '00000001')
            if (bin_to_int(reminder) < 0 and bin_to_int(reminder_prev) > 0) or \
                (bin_to_int(reminder) > 0 and bin_to_int(reminder_prev) < 0):
                break
            quotient = Add()(quotient, ((len(a[:16])-1)*'0')+'1')
        quotient = LeftShift()(quotient,4)
        return quotient+a[16:]
    
class Sqrt(Gate):
    
    def __call__(self,x):
        num = copy.copy(x[:16])
        xx = bin_to_int(x[:16])
        #e = [i for i in range(int(xx)) if pow(4,i)<=xx][-1]
        e = [i for i in range(int(xx)) if pow(4,i)<=xx]
        #e = e[-1] if e else 0
        e = int_to_bin(pow(4,e[-1])) if e else '0'*len(num)
        if len(e) < len(num):
            e = PadZerosLeft()(e,len(num) - len(e))
        r = '0'*len(e)
        while '1' in e:
            if GreaterThanEqualTo()(num, Add()(r,e)):
                num = Subtract()(num, Add()(r,e))
                r = Add()(RightShift()(r, 1), e)
            else:
                r = RightShift()(r, 1)
            e = RightShift()(e, 2)
        r = LeftShift()(r,2)
        return r + x[16:]
    
class GreaterThanEqualTo(Gate):
    def __call__(self,a,b):
        return bin_to_int(a) >= bin_to_int(b)
    
class PadZerosLeft(Gate):
    def __call__(self,a,n):
        return n*'0' + a 

class PadZerosRight(Gate):
    def __call__(self,a,n):
        return a + n*'0'
    
class GarbledCircuit(object):
    def __init__(A,b):
        self.A = A
        self.b = b
        
    def generate_beta(self):
        pass

In [12]:
def bin_to_int_garbled(binary,wire_labels):
    wl = dict(zip(wire_labels.values(),wire_labels.keys()))
    b2 = ''.join([wl[i].split('_')[-1] for i in binary])
    int_part = 0
    for i,j in enumerate(reversed(b2[1:])):
        int_part += (int(j) * pow(2, i))
    int_part -= int(wl[binary[0]].split('_')[-1])*pow(2,len(binary)-1)
    return int_part

def compute_twos_complement_garbled(binary,wire_labels={}):
    wl = dict(zip(wire_labels.values(),wire_labels.keys()))
    out = []
    for i in binary:
        out += [wire_labels['B_1']] if '0' in wl[i] else [wire_labels['B_0']]
    #print(out)
    out = GarbledAdd()(((len(binary)-1)*[wire_labels['A_0']])+[wire_labels['A_1']], out, wire_labels)
    wl = dict(zip(wire_labels.values(),wire_labels.keys()))
    out = [wire_labels[wl[i].replace('C','B')] for i in out]
    return out

class GarbledLeftShift(Gate):
    def __call__(self,a, n, wire_labels={}, garbled_table=[]):
        #print(a,wire_labels)
        wl = dict(zip(wire_labels.values(), wire_labels.keys()))
        val =  [j for j in [wl[i] for i in list(set(a))] if '0' in j][0]
        return a[n:] + [wire_labels[val]] * n

class GarbledRightShift(Gate):
    def __call__(self,a, n, wire_labels={}, garbled_table=[]):
        wl = dict(zip(wire_labels.values(), wire_labels.keys()))
        val =  [j for j in [wl[i] for i in list(set(a))] if '0' in j][0]
        return [wire_labels[val]] * n + a[:-n]

class GarbledGreaterThanEqualTo(Gate):
    def __call__(self,a, b, wire_labels):
        return bin_to_int_garbled(a, wire_labels) >= bin_to_int_garbled(b, wire_labels)
    
class GarbledPadZerosLeft(Gate):
    def __call__(self,a, n, wire_labels={}, garbled_table=[]):
        wl = dict(zip(wire_labels.values(), wire_labels.keys()))
        val =  [j for j in [wl[i] for i in list(set(a))] if '0' in j][0]
        return [wire_labels[val]] * n + a 

class GarbledPadZerosRight(Gate):
    def __call__(self,a, n, wire_labels={}, garbled_table=[]):
        wl = dict(zip(wire_labels.values(), wire_labels.keys()))
        val =  [j for j in [wl[i] for i in list(set(a))] if '0' in j][0]
        return a + [wire_labels[val]] * n
    
class GarbledAnd(Gate):
    def __call__(self,a,b,wire_labels={},garbled_table=[]):
        
        des_b_0 = DES.new(wire_labels['B_0'], DES.MODE_ECB)
        des_a_0 = DES.new(wire_labels['A_0'], DES.MODE_ECB)
        des_b_1 = DES.new(wire_labels['B_1'], DES.MODE_ECB)
        des_a_1 = DES.new(wire_labels['A_1'], DES.MODE_ECB)
        ## 0 AND 0 = 0
        
        garbled_table.append(des_a_0.encrypt(des_b_0.encrypt(wire_labels['C_0'])))
        garbled_table.append(des_a_0.encrypt(des_b_1.encrypt(wire_labels['C_0'])))
        garbled_table.append(des_a_1.encrypt(des_b_0.encrypt(wire_labels['C_0'])))
        garbled_table.append(des_a_1.encrypt(des_b_1.encrypt(wire_labels['C_1'])))
        
        des_in = DES.new(b, DES.MODE_ECB)
        des_out = DES.new(a, DES.MODE_ECB)
        for g in garbled_table:
            try:
                deciphered = des_in.decrypt(des_out.decrypt(g)).decode()
            except UnicodeDecodeError:
                continue
            if deciphered in [ wire_labels['C_0'], wire_labels['C_1'] ]:
                return g
        
class GarbledOr(Gate):
    def __call__(self,a,b,wire_labels={},garbled_table=[]):
        
        des_b_0 = DES.new(wire_labels['B_0'], DES.MODE_ECB)
        des_a_0 = DES.new(wire_labels['A_0'], DES.MODE_ECB)
        des_b_1 = DES.new(wire_labels['B_1'], DES.MODE_ECB)
        des_a_1 = DES.new(wire_labels['A_1'], DES.MODE_ECB)
        ## 0 AND 0 = 0
        
        garbled_table.append(des_a_0.encrypt(des_b_0.encrypt(wire_labels['C_0'])))
        garbled_table.append(des_a_0.encrypt(des_b_1.encrypt(wire_labels['C_1'])))
        garbled_table.append(des_a_1.encrypt(des_b_0.encrypt(wire_labels['C_1'])))
        garbled_table.append(des_a_1.encrypt(des_b_1.encrypt(wire_labels['C_1'])))
        
        des_in = DES.new(b, DES.MODE_ECB)
        des_out = DES.new(a, DES.MODE_ECB)
        for g in garbled_table:
            try:
                deciphered = des_in.decrypt(des_out.decrypt(g)).decode()
            except UnicodeDecodeError:
                continue
            if deciphered in [ wire_labels['C_0'], wire_labels['C_1'] ]:
                return g

class GarbledXor(Gate):
    def __call__(self,a,b,wire_labels={},garbled_table=[]):
        
        des_b_0 = DES.new(wire_labels['B_0'], DES.MODE_ECB)
        des_a_0 = DES.new(wire_labels['A_0'], DES.MODE_ECB)
        des_b_1 = DES.new(wire_labels['B_1'], DES.MODE_ECB)
        des_a_1 = DES.new(wire_labels['A_1'], DES.MODE_ECB)
        ## 0 AND 0 = 0
        
        garbled_table.append(des_a_0.encrypt(des_b_0.encrypt(wire_labels['C_0'])))
        garbled_table.append(des_a_0.encrypt(des_b_1.encrypt(wire_labels['C_1'])))
        garbled_table.append(des_a_1.encrypt(des_b_0.encrypt(wire_labels['C_1'])))
        garbled_table.append(des_a_1.encrypt(des_b_1.encrypt(wire_labels['C_0'])))
        #print(garbled_table)
        des_1 = DES.new(a, DES.MODE_ECB)
        des_2 = DES.new(b, DES.MODE_ECB)
        for g in garbled_table:
            try:
                deciphered = des_2.decrypt(des_1.decrypt(g)).decode()
            except UnicodeDecodeError:
                continue
            #print(deciphered)
            #print([ wire_labels['C_0'], wire_labels['C_1'] ])
            if deciphered in [ wire_labels['C_0'], wire_labels['C_1'] ]:
                return g
        
class GarbledNot(Gate):
    def __call__(self,b,wire_labels={},garbled_table=[]):
        
        des_a_0 = DES.new(wire_labels['A_0'], DES.MODE_ECB)
        des_a_1 = DES.new(wire_labels['A_1'], DES.MODE_ECB)
        
        garbled_table.append(des_a_0.encrypt(wire_labels['C_1']))
        garbled_table.append(des_a_1.encrypt(wire_labels['C_0']))
        
        des = DES.new(b, DES.MODE_ECB)
        for g in garbled_table:
            try:
                deciphered = des.decrypt(g).decode()
            except UnicodeDecodeError:
                continue
            if deciphered in [ wire_labels['C_0'], wire_labels['C_1'] ]:
                return g
            
class GarbledAdd(Gate):
    
    def add(self,a,b,carry,wire_labels):
        wl = dict(zip(wire_labels.values(), wire_labels.keys()))
        
        des_in = DES.new(b, DES.MODE_ECB)
        des_out = DES.new(a, DES.MODE_ECB)
        
        o1 = GarbledXor()(a,b,wire_labels)
        #print(a,b,o1)
        o1 = des_in.decrypt(des_out.decrypt(o1)).decode()
        
        o2 = GarbledAnd()(a,b,wire_labels)
        o2 = des_in.decrypt(des_out.decrypt(o2)).decode()
        
        o1 = wire_labels[wl[o1].replace('C','B')]
        
        des_in = DES.new(o1, DES.MODE_ECB)
        des_out = DES.new(carry, DES.MODE_ECB)
        
        add = GarbledXor()(carry,o1,wire_labels)
        add = des_in.decrypt(des_out.decrypt(add)).decode()
        
        o4 = GarbledAnd()(carry,o1,wire_labels)
        o4 = des_in.decrypt(des_out.decrypt(o4)).decode()
        
        o4 = wire_labels[wl[o4].replace('C', 'A')]
        o2 = wire_labels[wl[o2].replace('C', 'B')]
        
        des_in = DES.new(o2, DES.MODE_ECB)
        des_out = DES.new(o4, DES.MODE_ECB)
        
        cary = GarbledOr()(o4, o2, wire_labels)
        cary = des_in.decrypt(des_out.decrypt(cary)).decode()
        
        cary = wire_labels[wl[cary].replace('C', 'A')]
        return str(add), cary
    
    def __call__(self,a, b, wire_labels={}):
        wl = dict(zip(wire_labels.values(),wire_labels.keys()))
        carry = wire_labels['A_0']
        out = []
        for idx,i in reversed(list(enumerate(zip(a[:16],b[:16])))):
            o,carry = self.add(i[0], i[1], carry, wire_labels)
            out.append(o)
        #print(out)
        out = list(reversed(out))
        return out + [ wire_labels[wl[i].replace('A','C')] for i in a[16:]]

class GarbledSubtract(Gate):
    
    def __call__(self,a,b, wire_labels={}):
        b2 = compute_twos_complement_garbled(b[:16],wire_labels)
        b2 = b2 + b[16:]
        return GarbledAdd()(a,b2,wire_labels)

class GarbledMultiplication(Gate):

    def __call__(self,a,b, wire_labels={}):
        wl = dict(zip(wire_labels.values(),wire_labels.keys()))
        s = [wire_labels['B_0']]*16
        m = copy.copy(a[:16])
        for i in reversed(b[:16]):
            #print(m,s)
            if '1' in wl[i]:     # when digit is one, add the intermediary product
                s = GarbledAdd()(m, s, wire_labels)
                s = [ wire_labels[wl[i].replace('C','B')] for i in s]
            m = GarbledLeftShift()(m, 1, wire_labels)  # shift one per digit in b
        #exponent = Add()(a[16:],b[16:])
        s = [ wire_labels[wl[i].replace('B','C')] for i in s]
        s = GarbledRightShift()(s,4,wire_labels) ## Shifting left by 4 places to get exponent back to 2^-4
        return s+b[16:]
        
    def __call__(self,a, b):
        reminder,reminder_prev = Subtract()(a[:16], b[:16]), copy.copy(a[:16])
        quotient = Add()('0'*len(a[:16]), ((len(a[:16])-1)*'0')+'1')
        while '1' in reminder:
            reminder_prev = copy.copy(reminder)
            reminder = Subtract()(reminder, b[:16])
            #print(bin_to_int(reminder))
            #quotient = Add()(quotient, '00000001')
            if (bin_to_int(reminder) < 0 and bin_to_int(reminder_prev) > 0) or \
                (bin_to_int(reminder) > 0 and bin_to_int(reminder_prev) < 0):
                break
            quotient = Add()(quotient, ((len(a[:16])-1)*'0')+'1')
        quotient = LeftShift()(quotient,4)
        return quotient+a[16:]
    
class GarbledDivision(Gate):
    
    def __call__(self,a, b,wire_labels={}):
        wl = dict(zip(wire_labels.values(),wire_labels.keys()))
        reminder,reminder_prev = GarbledSubtract()(a[:16], b[:16], wire_labels), copy.copy(a[:16])
        quotient = GarbledAdd()([wire_labels['A_0']]*16, ([wire_labels['B_0']] * 15)+[wire_labels['B_1']], wire_labels)
        reminder = [ wire_labels[wl[val].replace('C', 'A')] for val in reminder]
        while wire_labels['A_1'] in reminder:
            reminder_prev = copy.copy(reminder)
            reminder = GarbledSubtract()(reminder, b[:16], wire_labels)
            reminder = [ wire_labels[wl[val].replace('C', 'A')] for val in reminder]
            #print(bin_to_int(reminder))
            #quotient = Add()(quotient, '00000001')
            if (bin_to_int_garbled(reminder,wire_labels) < 0 and bin_to_int_garbled(reminder_prev,wire_labels) > 0) or \
                (bin_to_int_garbled(reminder,wire_labels) > 0 and bin_to_int_garbled(reminder_prev,wire_labels) < 0):
                break
            quotient = [ wire_labels[wl[val].replace('C', 'A')] for val in quotient]
            quotient = GarbledAdd()(quotient, ([wire_labels['B_0']] * 15)+[wire_labels['B_1']], wire_labels)
        quotient = GarbledLeftShift()(quotient,4, wire_labels)
        return quotient+[ wire_labels[wl[val].replace('A', 'C')] for val in a[16:]]
    
class GarbledSqrt(Gate):
    
    def __call__(self,x,wire_labels={}):
        wl = dict(zip(wire_labels.values(), wire_labels.keys()))
        num = copy.copy(x[:16])
        xx = bin_to_int_garbled(x[:16],wire_labels)
        #e = [i for i in range(int(xx)) if pow(4,i)<=xx][-1]
        e = [i for i in range(int(xx)) if pow(4,i)<=xx]
        #e = e[-1] if e else 0
        e = int_to_bin(pow(4,e[-1])) if e else '0'*len(num)
        e = [wire_labels['B_'+str(i)] for i in e]
        if len(e) < len(num):
            e = GarbledPadZerosLeft()(e,len(num) - len(e),wire_labels)
        r = [ wire_labels['A_0'] ]*len(e)
        while wire_labels['B_1'] in e:
            #temp = Add()(r, e, wire_labels)
            #temp = [ wire_labels[wl[i].replace('C','A')] for i in temp]
            #print(r,e)
            if GarbledGreaterThanEqualTo()(num, GarbledAdd()(r, e, wire_labels), wire_labels):
                #print('IF')
                out = GarbledAdd()(r, e, wire_labels)
                out = [ wire_labels[wl[i].replace('C','B')] for i in out]
                num = GarbledSubtract()(num, out,wire_labels)
                num = [ wire_labels[wl[i].replace('C','A')] for i in num]
                r = GarbledAdd()(GarbledRightShift()(r, 1, wire_labels), e, wire_labels)
                #print(r)
                r = [ wire_labels[wl[i].replace('C','A')] for i in r]
                #print(r)
            else:
                #print('ELSE')
                r = GarbledRightShift()(r, 1, wire_labels)
            e = GarbledRightShift()(e, 2, wire_labels)
        r = GarbledLeftShift()(r,2, wire_labels)
        return r + [ wire_labels[wl[i].replace('A','C')] for i in x[16:] ]
    
class GarbledCircuit(object):
    def __init__(A,b):
        self.A = A
        self.b = b
        
    def generate_beta(self):
        pass

In [128]:
def poc_AddSubtractDivMultiSqrt():
    add = Add()
    print(add('010','011'))
    print(Add()('010','011'))
    print(Subtract()('010','001'))
    print(Subtract()('111','010'))
    print(Add()('111','010'))
    print(bin_to_float(Add()(float_to_bin(1.5),float_to_bin(1.5))))
    print(bin_to_float(float_to_bin(1.5)))
    print(bin_to_float(float_to_bin(1.8)))
    print(compute_twos_complement('0001'))
    print(Multiplication()('1110','0001'))
    print(Sqrt()('00010000'))
    print(Sqrt()('00010000'))
    print(Sqrt()('000100'))
    print(Sqrt()('00100000'))
    print(Sqrt()('0101'))
    print(Division()('00000111','00000011')) 
    print(Division()('10000000','10000000'))
    print(Division()('10000000','00000001'))
    print(Division()('00000001','11111111')) ## Wrong
    #print(Subtract()('10000000','00000001'))

In [56]:
#poc_AddSubtractDivMultiSqrt()

In [57]:
'https://www.electrical4u.com/2s-complement-arithmetic/'

'https://www.electrical4u.com/2s-complement-arithmetic/'

In [58]:
def cholesky_decomposition_binary(A):
    d = A.shape[0]
    L = A.copy()
    #L = np.zeros(A.shape)
    for i in range(d-1):
        for j in range(i+1,d):
            L[i][j] = '0'*24
    for j in range(d):
        for k in range(j-1):
            for i in range(j,d):
                out = Multiplication()(L[i][k], L[j][k])
                L[i][j] = Subtract()(L[i][j], out) ## Need to bring -4 as exponent
        L[j][j] = Sqrt()(L[j][j]) ## Need to bring -4 as exponent
        for k in range(j+1,d):
            L[k][j] = Division()(L[k][j],L[j][j]) ## Need to bring -4 as exponent
    return L

In [59]:
A2 = np.empty(A.shape,dtype=np.object)
for i in range(A.shape[0]):
    for j in range(A.shape[1]):
        A2[i][j] = float_to_bin(A[i][j])

In [60]:
#A2

In [61]:
#A3 = np.empty(A2.shape,dtype=np.object)
#for i in range(A2.shape[0]):
#    for j in range(A2.shape[1]):
#        A3[i][j] = bin_to_float(A2[i][j])

In [62]:
#A,A3

In [63]:
%time ans = Add()('000000000110000011111100', '000000000011000011111100')
print(bin_to_int('0000000000110000'),bin_to_int('0000000001100000'),bin_to_int(ans[:16]))

CPU times: user 180 µs, sys: 13 µs, total: 193 µs
Wall time: 199 µs
48 96 144


In [64]:
%time ans = Subtract()(compute_twos_complement('000000000110000011111100'), compute_twos_complement('000000000011000011111100'))
print(bin_to_int('0000000001100000'),bin_to_int('0000000000110000'),bin_to_int(ans[:16]))

CPU times: user 667 µs, sys: 46 µs, total: 713 µs
Wall time: 721 µs
96 48 -48


In [65]:
%time ans = Multiplication()('000000000110000011111100', '000000000011000011111100')
print(bin_to_float('000000000110000011111100'),bin_to_float('000000000011000011111100'),bin_to_float(ans))

CPU times: user 458 µs, sys: 0 ns, total: 458 µs
Wall time: 427 µs
6.0 3.0 18.0


In [66]:
%time ans = Division()('000000000110000011111100', '000000000011000011111100')
print(bin_to_float('000000000110000011111100'),bin_to_float('000000000011000011111100'),bin_to_float(ans))

CPU times: user 1 ms, sys: 70 µs, total: 1.07 ms
Wall time: 1.08 ms
6.0 3.0 2.0


In [67]:
%time ans = Sqrt()('000000000110000011111100')
print(bin_to_float('000000000110000011111100'),bin_to_float(ans))

CPU times: user 2.7 ms, sys: 190 µs, total: 2.89 ms
Wall time: 2.78 ms
6.0 2.25


In [68]:
%time L2 = cholesky_decomposition_binary(A2)

CPU times: user 535 ms, sys: 3.49 ms, total: 538 ms
Wall time: 546 ms


In [69]:
L3 = np.empty(L2.shape,dtype=np.object)
for i in range(L2.shape[0]):
    for j in range(L2.shape[1]):
        L3[i][j] = bin_to_float(L2[i][j])

In [70]:
L3,cholesky(A),cholesky_decomposition(A)

(array([[2.25, 0.0, 0.0, 0.0],
        [1.0, 2.25, 0.0, 0.0],
        [1.0, 2.0, 3.0, 0.0],
        [3.0, 910.0, 1.0, 0.0]], dtype=object),
 array([[ 2.44948974,  0.        ,  0.        ,  0.        ],
        [ 1.22474487,  2.12132034,  0.        ,  0.        ],
        [ 1.63299316,  1.41421356,  2.30940108,  0.        ],
        [ 3.26598632, -1.41421356,  1.58771324,  3.13249102]]),
 array([[2.4494898 , 0.        , 0.        , 0.        ],
        [1.2247448 , 2.4494898 , 0.        , 0.        ],
        [1.6329931 , 2.0412414 , 2.7080128 , 0.        ],
        [3.2659862 , 0.40824828, 0.6154576 , 3.7638633 ]], dtype=float32))

In [71]:
np.dot(L3,L3.T), A

(array([[5.0625, 2.25, 2.25, 6.75],
        [2.25, 6.0625, 5.5, 2050.5],
        [2.25, 5.5, 14.0, 1826.0],
        [6.75, 2050.5, 1826.0, 828110.0]], dtype=object),
 array([[ 6.,  3.,  4.,  8.],
        [ 3.,  6.,  5.,  1.],
        [ 4.,  5., 10.,  7.],
        [ 8.,  1.,  7., 25.]], dtype=float32))

In [72]:
def back_substitution_lower_np(L,Y):
    beta = np.empty(Y.shape,dtype=np.float32)
    beta[0] = Y[0] / L[0][0]
    for i in range(1,len(beta)):
        s = 0.0
        for j in range(i):
            mul = L[i][j] * beta[j]
            # print(i,j,L[i][j],beta[j],mul)
            s += mul
        #print(Y[i] / s)
        beta[i] = Y[i] / s
    return beta

In [73]:
def back_substitution_garbled(L,Y):
    beta = np.empty(Y.shape,dtype=np.float32)
    beta[0] = Divide()(Y[0], L[0][0])
    for i in range(1,len(beta)):
        s = '0'*16 + '00000000'
        for j in range(i):
            mul = Multiplication()(L[i][j], beta[j])
            s = Add()(s, mul)
        beta[i] = Divide()(Y[i], s)
    return beta

In [74]:
%time back_substitution_lower_np(cholesky(A),np.array([1.,2.,3.,4.]))

CPU times: user 464 µs, sys: 32 µs, total: 496 µs
Wall time: 502 µs


array([ 0.4082483 ,  3.9999998 ,  0.47441927, -1.1203607 ], dtype=float32)

In [75]:
pickle.loads(pickle.dumps(np.arange(5)))

array([0, 1, 2, 3, 4])

In [76]:
#pickle.dumps(np.arange(5))

In [32]:
class User(object):
    def __init__(self):
        df = pd.read_csv('auto-mpg.csv')
        df = df[df['horsepower'] != '?']
        cols = df.columns.tolist()
        cols.remove('mpg')
        cols.remove('car name')
        self.X = df[cols].values.astype(np.float32)
        self.y = df['mpg'].values.astype(np.float32)
        self.X_train,self.X_test,self.y_train,self.y_test = train_test_split(self.X,self.y,train_size=.8,test_size=.2)
        sc = StandardScaler()
        self.X_train = sc.fit_transform(self.X_train)
        self.X_test = sc.transform(self.X_test)
        self.X_train = np.around(self.X_train,2)
        self.X_test = np.around(self.X_test,2)
        #self.temp = np.around(np.dot(self.X_train.T,self.X_train),2)
        #print(np.around(np.dot(self.X_train.T,self.X_train),2))
    
    def paillier_encrypt(self, A_i, b_i, precision=2):
        A_i_enc = np.zeros(A_i.shape,dtype=np.object)
        for i in range(A_i.shape[0]):
            for j in range(A_i.shape[1]):
                #print(A_i[i][j])
                A_i_enc[i][j] = self.public_key.encrypt(A_i[i][j],precision=precision)
                
        b_i_enc = np.zeros(b_i.shape,dtype=np.object)
        for i in range(b_i.shape[0]):
            b_i_enc[i] = self.public_key.encrypt(b_i[i],precision=precision)
        
        return A_i_enc, b_i_enc
    
    def calculate(self):
        self.c = []
        #print(self.X_train.shape,self.y_train.shape)
        for x_i,y_i in zip(self.X_train,self.y_train):
            #print(x_i, y_i)
            A_i = np.dot(x_i.reshape(len(x_i), 1), x_i.reshape(len(x_i), 1).T)
            b_i = x_i * y_i
            #print(A_i.shape, b_i.shape)
            c_i = self.paillier_encrypt(A_i, b_i, precision=2)
            self.c.append(c_i)
        #c_mu = self.paillier_encrypt(self.mu_A, self.mu_b, precision=2)
        #self.c.append(c_mu)
        return self.c

In [ ]:
class CSP(object):
    def __init__(self):
        self.public_key, self.private_key = paillier.generate_paillier_keypair()
        self.mu_Af, self.mu_bf = np.random.randn((7,7)), np.random.randn(7)
    
    def paillier_decrypt(self,c_i,precision=2):
        #c = user.calculate_and_send_to_evaluater()
        A_hat, b_hat = np.empty(c_i[0].shape, dtype=np.object), np.empty(c_i[1].shape, dtype=np.object)
        for i in range(c_i[0].shape[0]):
            for j in range(c_i[0].shape[0]):
                A_hat[i][j] = self.private_key.decrypt(c_i[0][i][j])

        for j in range(c_i[1].shape[0]):
                b_hat[j] = self.private_key.decrypt(c_i[1][j])
        
        return A_hat, b_hat
    
    def garble_input(self):
        wire_labels = {}
        wire_labels['A_0'] = token_hex(4)
        wire_labels['A_1'] = token_hex(4)
        wire_labels['B_0'] = token_hex(4)
        wire_labels['B_1'] = token_hex(4)
        wire_labels['C_0'] = token_hex(4)
        wire_labels['C_1'] = token_hex(4)
        
        #self.c_i = self.sock.receive()
        
        self.A_hat, self.b_hat = self.paillier_decrypt(self.c_i)
        
        #self.mu_A, self.mu_b = np.empty(self.mu_Af.shape,dtype=np.object), np.empty(self.mu_bf.shape,dtype=np.object)
        
        for i in range(self.mu_A.shape[0]):
            for j in range(self.mu_A.shape[1]):
                self.mu_A[i][j] = float_to_bin(self.mu_Af[i][j])
        
        for i in range(self.mu_b.shape[0]):
            self.mu_b[i] = float_to_bin(self.mu_bf[i])
        
        A_hat_garbled = np.empty(self.A_hat.shape + (24,),dtype = np.object)
        b_hat_garbled = np.empty(self.b_hat.shape + (24,),dtype = np.object)
        mu_A_garbled = np.empty(self.A_hat.shape + (24,),dtype = np.object)
        mu_b_garbled = np.empty(self.b_hat.shape + (24,),dtype = np.object)
        
        for i in range(self.A_hat.shape[0]):
            for j in range(self.b_hat.shape[1]):
                A_hat_garbled[i][j] = np.array([(wire_labels['A_0'] if k=='0' else wire_labels['A_1']) for k in self.A_hat[i][j]])
        
        for i in range(self.b_hat.shape[0]):
            b_hat_garbled[i] = A_hat_garbled[i][j] = np.array([(wire_labels['A_0'] if k=='0' else wire_labels['A_1']) for k in self.b_hat[i]])
            
        for i in range(self.A_hat.shape[0]):
            for j in range(self.b_hat.shape[1]):
                mu_A_garbled[i][j] = np.array([(wire_labels['B_0'] if k=='0' else wire_labels['B_1']) for k in self.A_hat[i][j]])
        
        for i in range(self.b_har.shape[0]):
            mu_b_garbled[i] = A_hat_garbled[i][j] = np.array([(wire_labels['B_0'] if k=='0' else wire_labels['B_1']) for k in self.b_hat[i]])
        
        
        #return self.send_to_evaluator(A_hat_garbled,b_hat_garbled,mu_A_garbled,mu_b_garbled,wire_labels,mu_Af,mu_bf)
        return {
            'A_HAT_GARBLED':A_hat_garbled,
            'B_HAT_GARBLED': b_hat_garbled,
            'MU_A_GARBLED' : mu_A_garbled,
            'MU_B_GARBLED' : mu_b_garbled,
            'WIRE_LABELS'  : wire_labels,
            'PUBLIC KEY'   : self.public_key,
            'MU_AF'        : mu_Af,
            'MU_BF'        : mu_bf
         }
        
    def send_to_evaluator(self,A_hat_garbled,b_hat_garbled,mu_A_garbled,mu_b_garbled,wire_labels,mu_Af,mu_bf):
        return {
            'A_HAT_GARBLED':A_hat_garbled,
            'B_HAT_GARBLED': b_hat_garbled,
            'MU_A_GARBLED' : mu_A_garbled,
            'MU_B_GARBLED' : mu_b_garbled,
            'WIRE_LABELS'  : wire_labels,
            'PUBLIC KEY'   : self.public_key,
            'MU_AF'        : mu_Af,
            'MU_BF'        : mu_bf
        }

In [34]:
class Evaluator(object):
    def __init__(self):
        #self.A_hat = 
        pass
    
    def receive_c_hat_values(self):
        pass    
    
    def calculate_c_hat(self):
        As,bs = zip(*self.c)
        c_final = sum(As),sum(bs)
        c_final = c_final[0]+self.mu_Af,c_final[1]+self.mu_bf
        return c_final
    
    def send_c_hat_to_csp_for_decryption(self):
        pass
    
    def convert_to_float(self,a,two_dimension=True):
        out = np.empty(a.shape,dtype=np.float)
        wl = dict(zip(self.wire_labels.values(),wire_labels.keys()))
        if two_dimension:
            for i in range(a.shape[0]):
                for j in range(a.shape[1]):
                    out[i][j] = bin_to_float(''.join([ wl[k].split('_')[-1] for k in a[i][j] ]))
        else:
            for i in range(a.shape[0]):
                out[i] = bin_to_float(''.join([ wl[k].split('_')[-1] for k in a[i] ]))
                
        return out
        
    def calculate_beta(self):
        ## Subtract mu_A and mu_B from A_hat and b_hat
        A_garbled = np.empty(self.A_hat_garbled.shape,dtype=np.object)
        b_garbled = np.empty(self.b_hat_garbled.shape,dtype=np.object)
        
        for i in range(self.A_hat_garbled.shape[0]):
            for j in range(self.A_hat_garbled.shape[1]):
                A_garbled[i][j] = np.array(GarbledSubtract(self.A_hat_garbled[i][j]),self.mu_A_garbled[i][j],self.wire_labels)
        
        for i in range(self.b_hat_garbled.shape[0]):
            b_garbled[i][j] = np.array(GarbledSubtract(self.b_hat_garbled[i],self.mu_b_garbled[i], self.wire_labels))
        
        print(self.convert_to_float(A_garbled, wire_labels))
        print(self.convert_to_float(b_garbled, wire_labels,False))
        ## Get L using A with algo cholesky
        wl = dict(zip(self.wire_labels.values(), self.wire_labels.keys()))
        d = A_garbled.shape[0]
        L = A_garbled.copy()
        for i in range(A_garbled.shape[0]):
            for j in range(A_garbled.shape[1]):
                L[i][j] = np.array([wire_labels(wl[i].replace('C','A')) for i in L[i][j]])
        #L = np.zeros(A.shape)
        for i in range(d-1):
            for j in range(i+1,d):
                L[i][j] = self.wire_labels['A_0']*24
        for j in range(d):
            for k in range(j-1):
                for i in range(j,d):
                    ljk = np.array([wire_labels(wl[i].replace('A','B')) for i in L[j][k]])
                    out = GarbledMultiplication()(L[i][k], L[j][k],self.wire_labels)
                    out = np.array([wire_labels(wl[i].replace('C','B')) for i in out])
                    L[i][j] = GarbledSubtract()(L[i][j], out) ## Need to bring -4 as exponent
                    L[i][j] = np.array([wire_labels(wl[i].replace('C','A')) for i in L[i][j]])
        
            L[j][j] = GarbledSqrt()(L[j][j],self.wire_labels) ## Need to bring -4 as exponent
            L[j][j] = np.array([wire_labels(wl[i].replace('C','A')) for i in L[j][j]])
            
            for k in range(j+1,d):
                ljj = np.array([self.wire_labels(wl[i].replace('A','B')) for i in L[j][j]])
                L[k][j] = GarbledDivision()(L[k][j], ljj, self.wire_labels) ## Need to bring -4 as exponent
                L[k][j] = np.array([self.wire_labels(wl[i].replace('C','A')) for i in L[k][j]])
        
        print(self.convert_to_float(L, wire_labels))
        ## Use back substitution algo to get individual values of Y    : L.T * Y = b
        LT = L.transpose(1,0,2)
        Y  = np.empty(self.b_garbled.shape,dtype=np.object)
        Y[d-1] = GarbledDivide()(b[d-1], LT[d-1][d-1], self.wire_labels)
        Y[d-1] = np.array([self.wire_labels(wl[i].replace('C','B')) for i in Y[d-1]])
        for i in reversed(range(len(d-1))):
            s = [ wire_labels['A_0'] ] * 24
            for j in range(i+1,d):
                temp_lij = np.array([self.wire_labels(wl[i].replace('C','A')) for i in LT[i][j]])
                mul = GarbledMultiplication()(temp_lij, Y[j], self.wire_labels)
                mul = np.array([self.wire_labels(wl[i].replace('C','B')) for i in mul])
                
                s = GarbledAdd()(s, mul, self.wire_labels)
                s = np.array([self.wire_labels(wl[i].replace('C','A')) for i in s])
                
            s = np.array([self.wire_labels(wl[i].replace('A','B')) for i in s])
            bi = np.array([self.wire_labels(wl[i].replace('C','A')) for i in b_garbled[i]])
            Y[i] = GarbledDivide()(bi, s, self.wire_labels)
            Y[i] = np.array([self.wire_labels(wl[i].replace('C','B')) for i in Y[i]])
          
        print(self.convert_to_float(Y, self.wire_labels))
        ## Use back substitution algo to get individual values of beta : L*beta = Y
        
        beta = np.empty(Y.shape,dtype=np.float32)
        beta[0] = GarbledDivide()(Y[0], L[0][0], self.wire_labels)
        beta[0] = np.array([self.wire_labels(wl[i].replace('C','B')) for i in beta[0]])
        for i in range(1,len(beta)):
            s = [ wire_labels['A_0'] ] * 24
            for j in range(i):
                temp_lij = np.array([self.wire_labels(wl[i].replace('C','A')) for i in L[i][j]])
                mul = Multiplication()(temp_lij, beta[j], self.wire_labels)
                mul = np.array([self.wire_labels(wl[i].replace('C','B')) for i in mul])
                
                s = GarbledAdd()(s, mul, self.wire_labels)
                s = np.array([self.wire_labels(wl[i].replace('C','A')) for i in s])
                
            s = np.array([self.wire_labels(wl[i].replace('A','B')) for i in s])
            yi = np.array([self.wire_labels(wl[i].replace('B','A')) for i in Y[i]])
            beta[i] = GarbledDivide()(yi, s, self.wire_labels)
            
        print(self.convert_to_float(beta, wire_labels))
        return beta, self.wire_labels

In [8]:
wire_labels = {}
wire_labels['A_0'] = token_hex(4)
wire_labels['A_1'] = token_hex(4)
wire_labels['B_0'] = token_hex(4)
wire_labels['B_1'] = token_hex(4)
wire_labels['C_0'] = token_hex(4)
wire_labels['C_1'] = token_hex(4)
a = '000000000110000011111100'
b = '000000000011000011111100'
a1 = [wire_labels['A_'+i] for i in a]
b1 = [wire_labels['B_'+i] for i in b]

%time ans = GarbledAdd()(a1, b1,wire_labels)
#print(bin_to_int('0000000000110000'),bin_to_int('0000000001100000'),bin_to_int(ans[:16]))

NameError: name 'token_hex' is not defined

In [81]:
wl = dict(zip(wire_labels.values(),wire_labels.keys()))
''.join([wl[i].split('_')[-1] for i in ans])
print(bin_to_int('0000000000110000'),bin_to_int('0000000001100000'),bin_to_int(''.join([wl[i].split('_')[-1] for i in ans])[:16]))

48 96 144


In [82]:
%time ans = GarbledSubtract()(a1, b1,wire_labels)
print(bin_to_int('0000000000110000'),bin_to_int('0000000001100000'),bin_to_int(''.join([wl[i].split('_')[-1] for i in ans])[:16]))

CPU times: user 48.2 ms, sys: 540 µs, total: 48.8 ms
Wall time: 50 ms
48 96 48


In [83]:
%time ans = GarbledSubtract()(a1, compute_twos_complement_garbled(b1,wire_labels),wire_labels)
print(bin_to_int('0000000000110000'),bin_to_int('0000000001100000'),bin_to_int(''.join([wl[i].split('_')[-1] for i in ans])[:16]))

CPU times: user 61.5 ms, sys: 3.83 ms, total: 65.3 ms
Wall time: 63.7 ms
48 96 145


In [85]:
%time ans = GarbledMultiplication()(a1, b1,wire_labels)
print(bin_to_float('000000000110000011111100'),bin_to_float('000000000011000011111100'),bin_to_float(''.join([wl[i].split('_')[-1] for i in ans])))

CPU times: user 39 ms, sys: 3.94 ms, total: 43 ms
Wall time: 42.9 ms
6.0 3.0 18.0


In [132]:
%time ans = GarbledDivision()(a1, b1,wire_labels)
print(bin_to_float('000000000110000011111100'),bin_to_float('000000000011000011111100'),bin_to_float(''.join([wl[i].split('_')[-1] for i in ans])))

CPU times: user 129 ms, sys: 0 ns, total: 129 ms
Wall time: 129 ms
6.0 3.0 2.0


In [133]:
%time ans = GarbledSqrt()(a1,wire_labels)
print(bin_to_float('000000000110000011111100'),bin_to_float('000000000011000011111100'),bin_to_float(''.join([wl[i].split('_')[-1] for i in ans])))

CPU times: user 254 ms, sys: 0 ns, total: 254 ms
Wall time: 257 ms
6.0 3.0 2.25


In [124]:
#wire_labels

In [3]:
(1,2) + (3,)

(1, 2, 3)

In [8]:
list(reversed(list(range(5))))

[4, 3, 2, 1, 0]

In [9]:
a = np.arange(6).reshape(1,2,3)

In [12]:
a.transpose(1,0,2).shape

(2, 1, 3)

In [35]:
user = User()

In [ ]:
csp = CSP()

In [ ]:
evaluator = Evaluator()

In [37]:
user.public_key = csp.public_key
c = user.calculate()

KeyboardInterrupt: 

In [ ]:
evaluator.c = c
evaluator.mu_Af = csp.mu_Af
evaluator.mu_bf = csp.mu_bf
c_final = evaluator.calculate_c_hat()

In [ ]:
csp.c_i = c_final
d = csp.garble_input()

In [ ]:
evaluator.A_hat_garbled = d['A_HAT_GARBLED']
evaluator.b_hat_garbled = d['B_HAT_GARBLED']
evaluator.mu_A_garbled = d['MU_A_GARBLED']
evaluator.mu_b_garbled = d['MU_B_GARBLED']
evaluator.wire_labels = d['WIRE_LABELS']

In [ ]:
evaluator.calculate_beta()